# Capstone Phase 2 - Modeling  
The previous notebook (capstone-phase1) was getting a bit long to run comfortably, so let's start a new one for ML.


#### Import some libraries we'll need

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from zipfile import ZipFile

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Load in the training and testing data sets.

In [2]:
with ZipFile('./Data/UNSW_NB15 training and testing sets.zip') as datasets_zip:
    # open each file and read in to a DataFrame
    with datasets_zip.open('UNSW_NB15_training-set.csv') as training_file:
        training_data = pd.read_csv(training_file, index_col='id')
    with datasets_zip.open('UNSW_NB15_testing-set.csv') as testing_file:
        testing_data = pd.read_csv(testing_file, index_col='id')

In [3]:
#training_data.shape

(175341, 44)

In [4]:
#testing_data.shape

(82332, 44)

## Apply transformations determined during exploration phase.

In [5]:
# split labels and categories from the training data
training = training_data.drop(['label', 'attack_cat'], axis=1)
training_labels = training_data['label'].copy()
training_cats = training_data['attack_cat'].copy()

# whatever we do to the training set should also be done to the testing set
testing = testing_data.drop(['label', 'attack_cat'], axis=1)
testing_labels = testing_data['label'].copy()
testing_cats = testing_data['attack_cat'].copy()

In [6]:
# expand categorical features
training_data_onehot = pd.get_dummies(training, columns=['proto', 'service', 'state'], prefix=['proto', 'service', 'state'])
#training_data_onehot.head()

# one-hot the testing set as well
testing_data_onehot = pd.get_dummies(testing, columns=['proto', 'service', 'state'], prefix=['proto', 'service', 'state'])

# convert target classes to binary vectors
binarizer = preprocessing.LabelBinarizer()
binarized_cats = binarizer.fit_transform(training_cats)
#print(binarizer.classes_)

# and for testing classes
test_binarizer = preprocessing.LabelBinarizer()
bin_test_cats = test_binarizer.fit_transform(testing_cats)
#print(test_binarizer.classes_)

['Analysis' 'Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
['Analysis' 'Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']


Let's also try a simple integer mapping for the classes, it might work better for some methods

In [7]:
y_train = training_cats.map({'Normal':0, 'Generic':1, 'Exploits':2, 'Fuzzers':3, 'DoS':4, 
                           'Reconnaissance':5, 'Analysis':6, 'Backdoor':7, 'Shellcode':8, 'Worms':9}).astype(int)
y_test = testing_cats.map({'Normal':0, 'Generic':1, 'Exploits':2, 'Fuzzers':3, 'DoS':4, 
                           'Reconnaissance':5, 'Analysis':6, 'Backdoor':7, 'Shellcode':8, 'Worms':9}).astype(int)
#y_test.head()

In [8]:
#print(training_data_onehot.shape)
#print(testing_data_onehot.shape)

(175341, 194)
(82332, 190)


The testing set doesn't appear to have all of the categorical values seen in the training set....

In [56]:
#missing_test_cols = [x for x in list(training_data_onehot.columns) if x not in list(testing_data_onehot.columns)]
#print(missing_test_cols)

['proto_icmp', 'proto_rtp', 'state_ECO', 'state_PAR', 'state_URN', 'state_no']


In [60]:
#print('Training protos: ', training['proto'].nunique())
#print('Testing protos: ', testing['proto'].nunique())
#print('Training services: ', training['service'].nunique())
#print('Testing services: ', testing['service'].nunique())
#print('Training states: ', training['state'].nunique())
#print('Testing states: ', testing['state'].nunique())

Training protos:  133
Testing protos:  131
Training services:  13
Testing services:  13
Training states:  9
Testing states:  7


In [64]:
#missing_protos = set(training['proto'].unique())-set(testing['proto'].unique())
#missing_states = set(training['state'].unique())-set(testing['state'].unique())
#print(missing_protos, missing_states)

{'icmp', 'rtp'} {'ECO', 'PAR', 'no', 'URN'}


In [63]:
#print(training['state'].unique())
#print(testing['state'].unique())

['FIN' 'INT' 'CON' 'ECO' 'REQ' 'RST' 'PAR' 'URN' 'no']
['INT' 'FIN' 'REQ' 'ACC' 'CON' 'RST' 'CLO']


It seems the training set doesn't have any instances of 'ACC' and 'CLO' states....

In [8]:
# apply standard scaler to the data
training_scaler = preprocessing.StandardScaler().fit(training_data_onehot)
training_data_standardized = training_scaler.transform(training_data_onehot)

# and the testing set
testing_scaler = preprocessing.StandardScaler().fit(testing_data_onehot)
testing_data_standardized = testing_scaler.transform(testing_data_onehot)

In [9]:
# convert standardized data back to DataFrame
training_data_std = pd.DataFrame(training_data_standardized, columns=training_data_onehot.columns)
# and testing data
testing_data_std = pd.DataFrame(testing_data_standardized, columns=testing_data_onehot.columns)

In [33]:
training_data_std.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_ssl,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,-0.191029,-0.104456,-0.135769,-0.049134,-0.102726,-0.576371,0.703839,1.578100,-0.389897,-0.273700,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
1,-0.109485,-0.046014,0.172599,-0.046410,0.188544,-0.576345,-1.141901,1.560002,-0.389928,-0.069233,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
2,0.040699,-0.089845,-0.026933,-0.048527,-0.012133,-0.576734,-1.141901,1.560002,-0.389964,-0.252044,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
3,0.049729,-0.060624,-0.063212,-0.047016,-0.098563,-0.576737,-1.141901,1.560002,-0.389958,-0.275821,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
4,-0.140417,-0.075235,-0.117630,-0.047554,-0.102057,-0.576617,0.723268,1.560002,-0.389927,-0.275561,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388


In [34]:
testing_data_std.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_FIN,state_INT,state_REQ,state_RST
0,-0.213727,-0.124455,-0.151816,-0.043684,-0.087369,0.057181,0.71944,-0.820395,0.643913,-0.263498,...,-0.018771,-0.049839,-0.019092,-0.00697,-0.003485,-0.304403,-0.956561,1.187424,-0.151277,-0.003485
1,-0.213728,-0.124455,-0.151816,-0.036308,-0.087369,0.286565,0.71944,-0.820395,4.539351,-0.263498,...,-0.018771,-0.049839,-0.019092,-0.00697,-0.003485,-0.304403,-0.956561,1.187424,-0.151277,-0.003485
2,-0.213729,-0.124455,-0.151816,-0.040351,-0.087369,0.791209,0.71944,-0.820395,4.391459,-0.263498,...,-0.018771,-0.049839,-0.019092,-0.00697,-0.003485,-0.304403,-0.956561,1.187424,-0.151277,-0.003485
3,-0.213729,-0.124455,-0.151816,-0.041330,-0.087369,0.566923,0.71944,-0.820395,2.977031,-0.263498,...,-0.018771,-0.049839,-0.019092,-0.00697,-0.003485,-0.304403,-0.956561,1.187424,-0.151277,-0.003485
4,-0.213728,-0.124455,-0.151816,-0.034187,-0.087369,0.118350,0.71944,-0.820395,4.369219,-0.263498,...,-0.018771,-0.049839,-0.019092,-0.00697,-0.003485,-0.304403,-0.956561,1.187424,-0.151277,-0.003485


## Load feature subsets

In [10]:
subsets = pd.read_pickle("./data/subsets.pkl")

In [12]:
subsets.head()

,Info Gain,ANOVA F-val,Var Thresh,MAD,Corr
0,sbytes,ct_dst_sport_ltm,spkts,sload,dur
1,smean,ct_dst_src_ltm,dpkts,rate,spkts
2,sload,ct_srv_dst,sbytes,dload,dpkts
3,dbytes,ct_src_dport_ltm,dbytes,sbytes,rate
4,rate,ct_srv_src,rate,dbytes,sttl


Make a subset of the Info Gain features

In [11]:
ig_cols = subsets['Info Gain'] # pandas Series
select_cols = training_data_std[ig_cols]

# test set
sel_test_cols = testing_data_std[ig_cols]

In [12]:
train_std_ig = select_cols.copy()
test_std_ig = sel_test_cols.copy()

In [41]:
train_std_ig.head()

,sbytes,smean,sload,dbytes,rate,dmean,dur,ct_dst_sport_ltm,ct_srv_dst,ct_state_ttl,dttl,dload,ct_dst_src_ltm,ct_srv_src,dinpkt,sttl,ct_src_dport_ltm,dpkts,ct_dst_ltm,sinpkt
0,-0.049134,-0.458048,-0.389897,-0.102726,-0.576371,-0.314240,-0.191029,-0.554373,-0.753074,-1.366486,1.578100,-0.273700,-0.705529,-0.775991,-0.080885,0.703839,-0.544736,-0.135769,-0.645013,-0.132788
1,-0.046410,-0.414076,-0.389928,0.188544,-0.576345,3.800869,-0.109485,-0.554373,-0.288257,-0.318711,1.560002,-0.069233,-0.614256,3.147666,-0.073735,-1.141901,-0.544736,0.172599,-0.645013,-0.129251
2,-0.048527,-0.443391,-0.389964,-0.012133,-0.576734,2.709185,0.040699,-0.554373,-0.288257,-0.318711,1.560002,-0.252044,-0.522983,-0.215468,0.014711,-1.141901,-0.544736,-0.026933,-0.520827,-0.104126
3,-0.047016,-0.414076,-0.389958,-0.098563,-0.576737,-0.232945,0.049729,-0.554373,-0.753074,-0.318711,1.560002,-0.275821,-0.522983,-0.775991,0.002046,-1.141901,-0.544736,-0.063212,-0.520827,-0.115034
4,-0.047554,-0.409190,-0.389927,-0.102057,-0.576617,-0.306498,-0.140417,-0.554373,2.779535,-0.318711,1.560002,-0.275561,2.854115,3.147666,-0.012721,0.723268,-0.420468,-0.117630,-0.520827,-0.129549


In [42]:
test_std_ig.head()

,sbytes,smean,sload,dbytes,rate,dmean,dur,ct_dst_sport_ltm,ct_srv_dst,ct_state_ttl,dttl,dload,ct_dst_src_ltm,ct_srv_src,dinpkt,sttl,ct_src_dport_ltm,dpkts,ct_dst_ltm,sinpkt
0,-0.043684,0.520319,0.643913,-0.087369,0.057181,-0.475371,-0.213727,-0.450186,-0.644190,0.591021,-0.820395,-0.263498,-0.477994,-0.680474,-0.094169,0.71944,-0.468312,-0.151816,-0.563660,-0.122179
1,-0.036308,3.556716,4.539351,-0.087369,0.286565,-0.475371,-0.213728,-0.450186,-0.644190,0.591021,-0.820395,-0.263498,-0.477994,-0.680474,-0.094169,0.71944,-0.468312,-0.151816,-0.563660,-0.122180
2,-0.040351,1.892214,4.391459,-0.087369,0.791209,-0.475371,-0.213729,-0.450186,-0.554273,0.591021,-0.820395,-0.263498,-0.390391,-0.590304,-0.094169,0.71944,-0.468312,-0.151816,-0.563660,-0.122180
3,-0.041330,1.489280,2.977031,-0.087369,0.566923,-0.475371,-0.213729,-0.450186,-0.554273,0.591021,-0.820395,-0.263498,-0.390391,-0.590304,-0.094169,0.71944,-0.349115,-0.151816,-0.444868,-0.122180
4,-0.034187,4.429740,4.369219,-0.087369,0.118350,-0.475371,-0.213728,-0.450186,-0.554273,0.591021,-0.820395,-0.263498,-0.390391,-0.590304,-0.094169,0.71944,-0.349115,-0.151816,-0.444868,-0.122179


## K-nearest neighbors

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [81]:
# start with default settings; k=5, distance=minkowski
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_model.fit(train_std_ig, y_train)#binarized_cats)

KNeighborsClassifier()

In [82]:
# predict classes for the test set
y_pred = knn_model.predict(test_std_ig)

Let's see how we did

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [86]:
con_mat = confusion_matrix(y_test, y_pred)#bin_test_cats, y_pred)
print(con_mat)

[[27061   368   991  7597    55   130   774    10    14     0]
 [  689 15887   408  1828    42     9     2     0     3     3]
 [  495   275  8044   682   924   264   252   187     9     0]
 [ 1768   197   667  2934   269    31    91   100     5     0]
 [  209   180  2036   229  1022    50   198   157     8     0]
 [  332    18   900   684   127  1400    20    14     1     0]
 [   55    86   264    29   123     1    69    50     0     0]
 [   47    82   228    42    99     5    36    44     0     0]
 [   47     9   117    89    12    75     0     0    29     0]
 [    3     1    25     6     2     0     0     0     0     7]]


In [87]:
class_rep = classification_report(y_test, y_pred)#bin_test_cats, y_pred)
print(class_rep)

              precision    recall  f1-score   support

           0       0.88      0.73      0.80     37000
           1       0.93      0.84      0.88     18871
           2       0.59      0.72      0.65     11132
           3       0.21      0.48      0.29      6062
           4       0.38      0.25      0.30      4089
           5       0.71      0.40      0.51      3496
           6       0.05      0.10      0.07       677
           7       0.08      0.08      0.08       583
           8       0.42      0.08      0.13       378
           9       0.70      0.16      0.26        44

    accuracy                           0.69     82332
   macro avg       0.49      0.38      0.40     82332
weighted avg       0.76      0.69      0.71     82332



I may need to rethink the Binarized Labels for this... Confusion Matrix and Classification Report probably need a simple integer mapping instead....

In [85]:
accuracy = accuracy_score(y_test, y_pred)#bin_test_cats, y_pred)
print(accuracy)

0.6862094932711461


68.62% with 5 neighbors, not great results

In [88]:
knn_model2 = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)
knn_model2.fit(train_std_ig, y_train)#binarized_cats)
y_pred2 = knn_model2.predict(test_std_ig)

In [90]:
con_mat = confusion_matrix(y_test, y_pred2)#bin_test_cats, y_pred)
print(con_mat)
class_rep = classification_report(y_test, y_pred2)#bin_test_cats, y_pred)
print(class_rep)
acc2 = accuracy_score(y_test, y_pred2)#bin_test_cats, y_pred)
print(acc2)

[[27017   395   830  7659    52   165   857     9    16     0]
 [  628 16189   413  1580    42    11     1     0     3     4]
 [  379   217  8389   678   768   326   242   119    13     1]
 [ 1683   197   722  3007   240    42    87    80     4     0]
 [  175   131  2378   242   830    74   167    86     6     0]
 [  136    16   748   781   103  1688    16     7     1     0]
 [   42    81   297    27   118     2    70    40     0     0]
 [   32    81   254    41    94     7    36    38     0     0]
 [   42    13   101    94     6    86     0     0    36     0]
 [    2     1    25     7     2     0     0     0     0     7]]
              precision    recall  f1-score   support

           0       0.90      0.73      0.80     37000
           1       0.93      0.86      0.89     18871
           2       0.59      0.75      0.66     11132
           3       0.21      0.50      0.30      6062
           4       0.37      0.20      0.26      4089
           5       0.70      0.48      0.57 

69.56% with 7 neighbors, very slighly better...maybe fewer will work better? or maybe we need significantly more?

In [91]:
knn_model3 = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn_model3.fit(train_std_ig, y_train)#binarized_cats)
y_pred3 = knn_model3.predict(test_std_ig)

In [92]:
con_mat = confusion_matrix(y_test, y_pred3)#bin_test_cats, y_pred)
print(con_mat)
class_rep = classification_report(y_test, y_pred3)#bin_test_cats, y_pred)
print(class_rep)
acc3 = accuracy_score(y_test, y_pred3)#bin_test_cats, y_pred)
print(acc3)

[[27078   370   964  7476    61   134   882    23    11     1]
 [  737 15970   323  1760    56    12     2     7     2     2]
 [  533   296  7303   931   743   573   305   425    20     3]
 [ 1812   204   579  3002   159    45   156    98     6     1]
 [  210   160  1690   406   792   162   200   457    12     0]
 [  259    19   544   879   269  1445    14    55     2    10]
 [   49    87   203    79    79     9   111    60     0     0]
 [   54    87   185    91    63     6    71    25     1     0]
 [   59    13    80    82    22    89     0     0    31     2]
 [    3     1    19     5     6     3     0     0     0     7]]
              precision    recall  f1-score   support

           0       0.88      0.73      0.80     37000
           1       0.93      0.85      0.89     18871
           2       0.61      0.66      0.63     11132
           3       0.20      0.50      0.29      6062
           4       0.35      0.19      0.25      4089
           5       0.58      0.41      0.48 

67.73% with 3 neighbors, this model probably just isn't well suited to this data set  
Let's try one more, there are 10 classes to predict, let's try 11 neighbors

In [93]:
knn_model4 = KNeighborsClassifier(n_neighbors=11, n_jobs=-1)
knn_model4.fit(train_std_ig, y_train)#binarized_cats)
y_pred4 = knn_model4.predict(test_std_ig)

In [94]:
con_mat = confusion_matrix(y_test, y_pred4)#bin_test_cats, y_pred)
print(con_mat)
class_rep = classification_report(y_test, y_pred4)#bin_test_cats, y_pred)
print(class_rep)
acc4 = accuracy_score(y_test, y_pred4)#bin_test_cats, y_pred)
print(acc4)

[[26929   371   842  7785    55   165   829     6    18     0]
 [  630 16136   442  1593    48    10     3     2     3     4]
 [  355   173  8420   724   872   280   186   107    14     1]
 [ 1596   178   721  3110   273    21   102    58     3     0]
 [  155   118  2326   251   985    58   102    89     5     0]
 [  127    10   710   775   135  1723     7     9     0     0]
 [   56    75   294    20   130     1    72    29     0     0]
 [   36    75   272    30    92     4    48    26     0     0]
 [   34    10    96    94    14    89     0     0    41     0]
 [    3     1    22     8     0     4     0     0     0     6]]
              precision    recall  f1-score   support

           0       0.90      0.73      0.80     37000
           1       0.94      0.86      0.90     18871
           2       0.60      0.76      0.67     11132
           3       0.22      0.51      0.30      6062
           4       0.38      0.24      0.29      4089
           5       0.73      0.49      0.59 

69.77%, again not much improvement
We either need different features, or more likely a more powerful model

Checking the results shows where the misclassificatons are happening most often:  
- Normal, ok  
- Generic, ok  
- Exploits, ok, highest misclass=DoS  
- Fuzzers, highest misclass=Normal  
- DoS, not ok, highest misclass=Exploits  
- Reconnaissance, not great, highest misclass=Exploits  
- Analysis, poor, highest misclass=Exploits, DoS  
- Backdoor, poor, highest miclass=Exploits  
- Shellcode, poor, highest misclass=Exploits, Fuzzers  
- Worms, poor, highest misclass=Exploits  

This is probably due to the relatively small number of some attack categories.  Those classes are often misclassified as *Exploits*.

### Let's try it with GridSeachCV

In [14]:
from sklearn.model_selection import GridSearchCV

https://medium.com/@erikgreenj/k-neighbors-classifier-with-gridsearchcv-basics-3c445ddeb657  
https://www.ritchieng.com/machine-learning-efficiently-search-tuning-param/

In [28]:
# since I've already run several tests using minkowski distance I'll leave those out, even though they weren't CV'd
params = {'n_neighbors':[5, 11, 19], 'weights':['uniform', 'distance'], 'metric':['euclidean', 'manhattan']}
knn_gs = GridSearchCV(KNeighborsClassifier(), params, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

In [29]:
knn_gs_results = knn_gs.fit(train_std_ig, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [31]:
print(knn_gs_results.best_score_)
print(knn_gs_results.best_estimator_)
print(knn_gs_results.best_params_)

0.7513416713717841
KNeighborsClassifier(metric='manhattan', n_neighbors=11)
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'uniform'}


75% accuracy looks decent, considering this isn't a very powerful model.

In [42]:
knn_gs_results.refit_time_

0.06899809837341309

Let's see all of the results

In [37]:
results = knn_gs_results.cv_results_
results_df = pd.DataFrame(results)
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.093810,0.000000,553.094102,8.798253,euclidean,5,uniform,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.797098,0.806748,0.625764,0.743203,0.083136,12
1,0.098164,0.003078,548.080443,2.152474,euclidean,5,distance,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.799100,0.810307,0.621520,0.743642,0.086474,11
2,0.163780,0.009183,543.430753,3.931821,euclidean,11,uniform,"{'metric': 'euclidean', 'n_neighbors': 11, 'we...",0.800811,0.808596,0.633976,0.747794,0.080544,7
3,0.132989,0.014661,545.358709,7.847026,euclidean,11,distance,"{'metric': 'euclidean', 'n_neighbors': 11, 'we...",0.803292,0.812326,0.626516,0.747378,0.085542,8
4,0.131787,0.007439,568.469995,14.267871,euclidean,19,uniform,"{'metric': 'euclidean', 'n_neighbors': 19, 'we...",0.798775,0.808271,0.629664,0.745570,0.082049,10
5,0.136094,0.012091,563.023459,14.878991,euclidean,19,distance,"{'metric': 'euclidean', 'n_neighbors': 19, 'we...",0.801666,0.811693,0.624822,0.746061,0.085826,9
6,0.130454,0.011011,731.575947,2.443129,manhattan,5,uniform,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",0.801495,0.807963,0.635636,0.748365,0.079755,5
7,0.116475,0.010368,724.358208,1.174405,manhattan,5,distance,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",0.803360,0.809879,0.630332,0.747857,0.083146,6
8,0.100615,0.000966,722.717544,3.753106,manhattan,11,uniform,"{'metric': 'manhattan', 'n_neighbors': 11, 'we...",0.804164,0.811658,0.638202,0.751342,0.080060,1
9,0.108833,0.009576,723.466347,4.693253,manhattan,11,distance,"{'metric': 'manhattan', 'n_neighbors': 11, 'we...",0.806851,0.814413,0.631033,0.750766,0.084720,3


In [43]:
type(knn_gs_results)

sklearn.model_selection._search.GridSearchCV

Let's save this for later.

In [45]:
import joblib

joblib.dump(knn_gs, './data/knn_grid_search.pkl') # load with knn_gs = joblib.load('./data/knn_grid_search.pkl')
joblib.dump(knn_gs_results, './data/knn_gs_results.pkl')

['./data/knn_gs_results.pkl']

In [46]:
knn_model_best = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan', n_jobs=-1)
knn_model_best.fit(train_std_ig, y_train)
y_pred_best = knn_model_best.predict(test_std_ig)

In [60]:
joblib.dump(knn_model_best, './data/knn_model_best_ig.pkl')

['./data/knn_model_best_ig.pkl']

In [49]:
con_mat_best = confusion_matrix(y_test, y_pred_best)
print(con_mat_best)
class_rep_best = classification_report(y_test, y_pred_best)
print(class_rep_best)
acc_best = accuracy_score(y_test, y_pred_best)
print(acc_best)

[[26955   328   917  7745    53   186   796     4    16     0]
 [  266 16959   428  1162    37     9     2     2     4     2]
 [  301   189  8685   524   923   311   147    37    14     1]
 [ 1505   196   823  3160   303    22    37    14     2     0]
 [  103   118  2481   179  1002    68   108    25     5     0]
 [   75    10   672   833   117  1771    11     6     1     0]
 [   27    83   366     3   148     1    43     6     0     0]
 [   14    83   332    15   112     6    13     8     0     0]
 [   30    12   102    91    11    86     0     0    46     0]
 [    3     0    27     8     0     0     0     0     0     6]]
              precision    recall  f1-score   support

           0       0.92      0.73      0.81     37000
           1       0.94      0.90      0.92     18871
           2       0.59      0.78      0.67     11132
           3       0.23      0.52      0.32      6062
           4       0.37      0.25      0.29      4089
           5       0.72      0.51      0.59 

Our best model achieved 71% on the Test Set.  Not too bad.

#### We should try the other feature sets.

In [33]:
# 'Info Gain', 'ANOVA F-val', 'Var Thresh', 'MAD', 'Corr'
anova_cols = subsets['ANOVA F-val'] # pandas Series
select_cols = training_data_std[anova_cols]
train_std_anova = select_cols.copy()
# test set
sel_test_cols = testing_data_std[anova_cols]
test_std_anova = sel_test_cols.copy()

varthrsh_cols = subsets['Var Thresh'] # pandas Series
select_cols = training_data_std[varthrsh_cols]
train_std_vt = select_cols.copy()
# test set
sel_test_cols = testing_data_std[varthrsh_cols]
test_std_vt = sel_test_cols.copy()

mad_cols = subsets['MAD'] # pandas Series
select_cols = training_data_std[mad_cols]
train_std_mad = select_cols.copy()
# test set
sel_test_cols = testing_data_std[mad_cols]
test_std_mad = sel_test_cols.copy()

corr_cols = subsets['Corr'] # pandas Series
select_cols = training_data_std[corr_cols]
train_std_corr = select_cols.copy()
# test set
sel_test_cols = testing_data_std[corr_cols]
test_std_corr = sel_test_cols.copy()

In [51]:
knn_model_best_anova = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan', n_jobs=-1)
knn_model_best_anova.fit(train_std_anova, y_train)
y_pred_best_anova = knn_model_best_anova.predict(test_std_anova)

In [61]:
joblib.dump(knn_model_best_anova, './data/knn_model_best_anova.pkl')

['./data/knn_model_best_anova.pkl']

In [55]:
con_mat_best_anova = confusion_matrix(y_test, y_pred_best_anova)
print(con_mat_best_anova)
class_rep_best_anova = classification_report(y_test, y_pred_best_anova)
print(class_rep_best_anova)
acc_best_best_anova = accuracy_score(y_test, y_pred_best_anova)
print(acc_best_best_anova)

[[25143   201  3503  7178   339   376   250     9     1     0]
 [  294 14367  2407   599  1129    55    17     1     0     2]
 [  345   518  7798   713   621   584   191   361     1     0]
 [ 1402   435   819  3023   198    66    40    79     0     0]
 [  160   346  1943   294   606   155   192   392     1     0]
 [  192    74  1529   618   216   790    23    54     0     0]
 [   90   208   214    45    69     2    13    36     0     0]
 [   60   206   171    65    46    11     8    16     0     0]
 [   21    10   150    76    17   103     0     1     0     0]
 [    4     1    22     6     0    10     0     0     0     1]]
              precision    recall  f1-score   support

           0       0.91      0.68      0.78     37000
           1       0.88      0.76      0.82     18871
           2       0.42      0.70      0.53     11132
           3       0.24      0.50      0.32      6062
           4       0.19      0.15      0.17      4089
           5       0.37      0.23      0.28 

In [52]:
knn_model_best_vt = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan', n_jobs=-1)
knn_model_best_vt.fit(train_std_vt, y_train)
y_pred_best_vt = knn_model_best_vt.predict(test_std_vt)

In [63]:
joblib.dump(knn_model_best_vt, './data/knn_model_best_vt.pkl')

['./data/knn_model_best_vt.pkl']

In [56]:
con_mat_best_vt = confusion_matrix(y_test, y_pred_best_vt)
print(con_mat_best_vt)
class_rep_best_vt = classification_report(y_test, y_pred_best_vt)
print(class_rep_best_vt)
acc_best_best_vt = accuracy_score(y_test, y_pred_best_vt)
print(acc_best_best_vt)

[[23799   576  4290  7252    73   893    62     0    55     0]
 [   69 10635   407  2414    27  1468     2     0  3848     1]
 [  427    36  7634  2094   221   516     9     0   195     0]
 [  859   238  1215  3150    61   330     2     0   207     0]
 [  112    52  1836  1424   224   336     1     0   104     0]
 [  168    13   877   746   260  1161     2     0   269     0]
 [   12     0   156   323    21    72     3     0    90     0]
 [    4     0   115   279    32    63     0     0    90     0]
 [   32     7   118   145     9    32     0     0    35     0]
 [    7     0    21    13     0     2     0     0     0     1]]
              precision    recall  f1-score   support

           0       0.93      0.64      0.76     37000
           1       0.92      0.56      0.70     18871
           2       0.46      0.69      0.55     11132
           3       0.18      0.52      0.26      6062
           4       0.24      0.05      0.09      4089
           5       0.24      0.33      0.28 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [53]:
knn_model_best_mad = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan', n_jobs=-1)
knn_model_best_mad.fit(train_std_mad, y_train)
y_pred_best_mad = knn_model_best.predict(test_std_mad)

In [64]:
joblib.dump(knn_model_best_mad, './data/knn_model_best_mad.pkl')

['./data/knn_model_best_mad.pkl']

In [57]:
con_mat_best_mad = confusion_matrix(y_test, y_pred_best_mad)
print(con_mat_best_mad)
class_rep_best_mad = classification_report(y_test, y_pred_best_mad)
print(class_rep_best_mad)
acc_best_best_mad = accuracy_score(y_test, y_pred_best_mad)
print(acc_best_best_mad)

[[19817  2869  5529  5871  2038    93   679   102     2     0]
 [ 7640  7271  1574  1487   893     1     5     0     0     0]
 [ 7879   160  2084   456   507     0    46     0     0     0]
 [ 2955   326  1169  1100   409     1    99     3     0     0]
 [ 2340   106   904   317   414     0     8     0     0     0]
 [ 3089     7   244    86    67     2     0     0     1     0]
 [  293    64   212    19    89     0     0     0     0     0]
 [  230    64   195    15    79     0     0     0     0     0]
 [  336     0    24    12     6     0     0     0     0     0]
 [   35     0     7     1     0     0     0     0     1     0]]
              precision    recall  f1-score   support

           0       0.44      0.54      0.49     37000
           1       0.67      0.39      0.49     18871
           2       0.17      0.19      0.18     11132
           3       0.12      0.18      0.14      6062
           4       0.09      0.10      0.10      4089
           5       0.02      0.00      0.00 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [54]:
knn_model_best_corr = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan', n_jobs=-1)
knn_model_best_corr.fit(train_std_corr, y_train)
y_pred_best_corr = knn_model_best_corr.predict(test_std_corr)

In [65]:
joblib.dump(knn_model_best_corr, './data/knn_model_best_corr.pkl')

['./data/knn_model_best_corr.pkl']

In [59]:
con_mat_best_corr = confusion_matrix(y_test, y_pred_best_corr)
print(con_mat_best_corr)
class_rep_best_corr = classification_report(y_test, y_pred_best_corr)
print(class_rep_best_corr)
acc_best_best_corr = accuracy_score(y_test, y_pred_best_corr)
print(acc_best_best_corr)

[[27460   583  1269  6992    92   233   311     2    58     0]
 [   91 17981   385   363    41     8     0     0     1     1]
 [  671    38  8825   566   706   232    72     9    13     0]
 [ 1412   164  1251  2881   237    71    21     1    24     0]
 [  286    52  2649   194   806    36    48    10     8     0]
 [  522    14   794   614   233  1307     5     2     5     0]
 [   50    14   501     3    99     2     8     0     0     0]
 [   25    14   426    18    85     4     8     3     0     0]
 [   78     9    56   157     9    24     0     1    44     0]
 [    2     0    28    10     1     0     0     0     0     3]]
              precision    recall  f1-score   support

           0       0.90      0.74      0.81     37000
           1       0.95      0.95      0.95     18871
           2       0.55      0.79      0.65     11132
           3       0.24      0.48      0.32      6062
           4       0.35      0.20      0.25      4089
           5       0.68      0.37      0.48 

OK, from these feature set only the set of correlated features approaches the accuracy found with the Info Gain set.  
71% accuracy on the Test Set using the top 20 Info Gain features.  
72% accuracy on the Test Set using the top 20 Correlation features.  
Should the hyperparameters be tuned for each feature subset, or is it OK to use the parameters found using the Info Gain set?

## Let's try a Naive Bayes classifier  
We'll try the Compliment Naive Bayes classifier, which "is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets."  
https://scikit-learn.org/stable/modules/naive_bayes.html

In [18]:
#from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB

In [19]:
#mnb_params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]}
#mnb_gs = GridSearchCV(MultinomialNB(), mnb_params, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

In [20]:
#mnb_gs_results = mnb_gs.fit(train_std_ig, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


ValueError: Negative values in data passed to MultinomialNB (input X)

Neither ComplementNB, nor MultinomialNB likes negative values.  Should I have used so other type of normalization instead of the Standard Scaler? Something that would generate only non-negative values? MinMaxScaler [0,1] "is often used as an alternative to zero mean, unit variance scaling"  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler

In [21]:
#train_std_ig.head()

,sbytes,smean,sload,dbytes,rate,dmean,dur,ct_dst_sport_ltm,ct_srv_dst,ct_state_ttl,dttl,dload,ct_dst_src_ltm,ct_srv_src,dinpkt,sttl,ct_src_dport_ltm,dpkts,ct_dst_ltm,sinpkt
0,-0.049134,-0.458048,-0.389897,-0.102726,-0.576371,-0.314240,-0.191029,-0.554373,-0.753074,-1.366486,1.578100,-0.273700,-0.705529,-0.775991,-0.080885,0.703839,-0.544736,-0.135769,-0.645013,-0.132788
1,-0.046410,-0.414076,-0.389928,0.188544,-0.576345,3.800869,-0.109485,-0.554373,-0.288257,-0.318711,1.560002,-0.069233,-0.614256,3.147666,-0.073735,-1.141901,-0.544736,0.172599,-0.645013,-0.129251
2,-0.048527,-0.443391,-0.389964,-0.012133,-0.576734,2.709185,0.040699,-0.554373,-0.288257,-0.318711,1.560002,-0.252044,-0.522983,-0.215468,0.014711,-1.141901,-0.544736,-0.026933,-0.520827,-0.104126
3,-0.047016,-0.414076,-0.389958,-0.098563,-0.576737,-0.232945,0.049729,-0.554373,-0.753074,-0.318711,1.560002,-0.275821,-0.522983,-0.775991,0.002046,-1.141901,-0.544736,-0.063212,-0.520827,-0.115034
4,-0.047554,-0.409190,-0.389927,-0.102057,-0.576617,-0.306498,-0.140417,-0.554373,2.779535,-0.318711,1.560002,-0.275561,2.854115,3.147666,-0.012721,0.723268,-0.420468,-0.117630,-0.520827,-0.129549


In [ ]:
#print(mnb_gs_results.best_score_)
#print(mnb_gs_results.best_estimator_)
#print(mnb_gs_results.best_params_)

#### Try Gaussian NB....

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

In [48]:
gnb = GaussianNB()
gnb.fit(train_std_ig, y_train)
cv_scores = cross_val_score(gnb, train_std_ig, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print('Cross validation scores: ', cv_scores)
print('Mean score: ', cv_scores.mean())
print('Standard Dev: ', cv_scores.std())

Cross validation scores:  [0.54914597 0.5327364  0.54639557 0.46823315 0.39842592]
Mean score:  0.498987400787217
Standard Dev:  0.0582423354572515


In [49]:
y_pred_gnb = gnb.predict(test_std_ig)
y_pred_gnb

array([7, 3, 3, ..., 0, 0, 5])

In [50]:
con_mat_gnb = confusion_matrix(y_test, y_pred_gnb)
print(con_mat_gnb)
class_rep_gnb = classification_report(y_test, y_pred_gnb)
print(class_rep_gnb)
acc_gnb = accuracy_score(y_test, y_pred_gnb)
print(acc_gnb)

[[14531   256  7283  3915    47  6784    29  4152     0     3]
 [   72 13821   217   138     4   179     0  4370     0    70]
 [  421   565  5012   535    15  1954     0  2415     0   215]
 [   51   612    42  1429     0  2270     0  1656     0     2]
 [   66   351   594   169    42   538     0  2280     0    49]
 [    2    24     8    39     0  2917     0   506     0     0]
 [    4   234    54     3     0     6     1   375     0     0]
 [    7   234    13     5     0    50     0   273     0     1]
 [    0     1     0    20     0   270     0    87     0     0]
 [    1     0     0     4     0    30     0     3     0     6]]
              precision    recall  f1-score   support

           0       0.96      0.39      0.56     37000
           1       0.86      0.73      0.79     18871
           2       0.38      0.45      0.41     11132
           3       0.23      0.24      0.23      6062
           4       0.39      0.01      0.02      4089
           5       0.19      0.83      0.32 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Try GaussianNB on the other feature sets

In [35]:
# ANOVA selected features
gnb_anova = GaussianNB()
gnb_anova.fit(train_std_anova, y_train)
cv_scores_anova = cross_val_score(gnb_anova, train_std_anova, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print('Cross validation scores: ', cv_scores_anova)
print('Mean score: ', cv_scores_anova.mean())
print('Standard Dev: ', cv_scores_anova.std())

Cross validation scores:  [0.57814594 0.56866659 0.56812479 0.50761378 0.33928368]
Mean score:  0.5123669547967329
Standard Dev:  0.09009536430085348


In [40]:
y_pred_gnb_anova = gnb_anova.predict(test_std_anova)
y_pred_gnb_anova

array([5, 5, 7, ..., 2, 2, 5])

In [44]:
con_mat_gnb_anova = confusion_matrix(y_test, y_pred_gnb_anova)
print(con_mat_gnb_anova)
class_rep_gnb_anova = classification_report(y_test, y_pred_gnb_anova)
print(class_rep_gnb_anova)
acc_gnb_anova = accuracy_score(y_test, y_pred_gnb_anova)
print(acc_gnb_anova)

[[15168  1153  6371  4186   772  6807     0  2483     0    60]
 [   83 15734   213    30   206   291     0  2286     0    28]
 [  502  1114  4995   121   257  2465     0  1609     0    69]
 [    0  1182    10  1798   102  2032     0   911     0    27]
 [   75   801   570    47   212   692     0  1665     0    27]
 [    1    73     6   122   121  2771     0   395     0     7]
 [    0   384    58     0    17     3     0   215     0     0]
 [    6   383    12     2    13    46     0   119     0     2]
 [    0     3     0    17    15   312     0    31     0     0]
 [    2     0     0     1     1    33     0     2     0     5]]
              precision    recall  f1-score   support

           0       0.96      0.41      0.57     37000
           1       0.76      0.83      0.79     18871
           2       0.41      0.45      0.43     11132
           3       0.28      0.30      0.29      6062
           4       0.12      0.05      0.07      4089
           5       0.18      0.79      0.29 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [36]:
# Variance Threshold selected features
gnb_vt = GaussianNB()
gnb_vt.fit(train_std_vt, y_train)
cv_scores_vt = cross_val_score(gnb_vt, train_std_vt, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print('Cross validation scores: ', cv_scores_vt)
print('Mean score: ', cv_scores_vt.mean())
print('Standard Dev: ', cv_scores_vt.std())

Cross validation scores:  [0.3518207  0.34897913 0.34915022 0.25039922 0.15381544]
Mean score:  0.29083294273584837
Standard Dev:  0.0786257849871457


In [41]:
y_pred_gnb_vt = gnb_vt.predict(test_std_vt)
y_pred_gnb_vt

array([1, 7, 7, ..., 0, 0, 1])

In [45]:
con_mat_gnb_vt = confusion_matrix(y_test, y_pred_gnb_vt)
print(con_mat_gnb_vt)
class_rep_gnb_vt = classification_report(y_test, y_pred_gnb_vt)
print(class_rep_gnb_vt)
acc_gnb_vt = accuracy_score(y_test, y_pred_gnb_vt)
print(acc_gnb_vt)

[[16638  2317  4580  2147   189  8170     0  2872     0    87]
 [  109 18231   139    68     7   159     0    97     0    61]
 [ 1212  2831  4078   340    46  1929     0   525     0   171]
 [   21  1789    50   847     3  2890     0   425     0    37]
 [   70  2596   526   108    22   295     0   401     0    71]
 [    3  1525     4    46     2  1813     0   103     0     0]
 [    4   590    54     3     0     0     0    26     0     0]
 [    0   484     8     3     0    37     0    49     0     2]
 [    0   160     0     8     0   187     0    23     0     0]
 [    0     4     0     1     0    31     0     1     0     7]]
              precision    recall  f1-score   support

           0       0.92      0.45      0.60     37000
           1       0.60      0.97      0.74     18871
           2       0.43      0.37      0.40     11132
           3       0.24      0.14      0.18      6062
           4       0.08      0.01      0.01      4089
           5       0.12      0.52      0.19 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [37]:
# Median Absolute Deviation selected features
gnb_mad = GaussianNB()
gnb_mad.fit(train_std_mad, y_train)
cv_scores_mad = cross_val_score(gnb_mad, train_std_mad, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print('Cross validation scores: ', cv_scores_mad)
print('Mean score: ', cv_scores_mad.mean())
print('Standard Dev: ', cv_scores_mad.std())

Cross validation scores:  [0.55156976 0.54035018 0.55754534 0.48984829 0.38405384]
Mean score:  0.5046734825495467
Standard Dev:  0.06486355791466777


In [42]:
y_pred_gnb_mad = gnb_mad.predict(test_std_mad)
y_pred_gnb_mad

array([7, 3, 3, ..., 0, 0, 5])

In [46]:
con_mat_gnb_mad = confusion_matrix(y_test, y_pred_gnb_mad)
print(con_mat_gnb_mad)
class_rep_gnb_mad = classification_report(y_test, y_pred_gnb_mad)
print(class_rep_gnb_mad)
acc_gnb_mad = accuracy_score(y_test, y_pred_gnb_mad)
print(acc_gnb_mad)

[[16263   306  4781  3742    64  6813    32  4922     0    77]
 [   87 13974   187   133     5   183     0  4209     0    93]
 [  581   652  4827   440    20  1866     0  2410     0   336]
 [   48   672    40  1388     8  2316     0  1575     0    15]
 [   72   449   548   148    41   461     0  2279     0    91]
 [    3    43     6    39     2  2801     0   602     0     0]
 [    3   246    53     3     0     4     1   367     0     0]
 [    6   246    12     5     2    48     0   262     0     2]
 [    0     3     0    19     0   252     0   104     0     0]
 [    0     0     0     3     0    29     0     4     0     8]]
              precision    recall  f1-score   support

           0       0.95      0.44      0.60     37000
           1       0.84      0.74      0.79     18871
           2       0.46      0.43      0.45     11132
           3       0.23      0.23      0.23      6062
           4       0.29      0.01      0.02      4089
           5       0.19      0.80      0.31 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [38]:
# Correlation selected features
gnb_corr = GaussianNB()
gnb_corr.fit(train_std_corr, y_train)
cv_scores_corr = cross_val_score(gnb_corr, train_std_corr, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print('Cross validation scores: ', cv_scores_corr)
print('Mean score: ', cv_scores_corr.mean())
print('Standard Dev: ', cv_scores_corr.std())

Cross validation scores:  [0.5430152  0.53427626 0.54149082 0.44462188 0.29938976]
Mean score:  0.4725587828493292
Standard Dev:  0.09411988263601356


In [43]:
y_pred_gnb_corr = gnb_corr.predict(test_std_corr)
y_pred_gnb_corr

array([3, 3, 3, ..., 4, 4, 3])

In [47]:
con_mat_gnb_corr = confusion_matrix(y_test, y_pred_gnb_corr)
print(con_mat_gnb_corr)
class_rep_gnb_corr = classification_report(y_test, y_pred_gnb_corr)
print(class_rep_gnb_corr)
acc_gnb_corr = accuracy_score(y_test, y_pred_gnb_corr)
print(acc_gnb_corr)

[[14614     0  5341 14354  2691     0     0     0     0     0]
 [ 6731     0   152  3310  8678     0     0     0     0     0]
 [ 1691     0  4052  3408  1981     0     0     0     0     0]
 [   26     0   186  5067   783     0     0     0     0     0]
 [   74     0   545  2342  1128     0     0     0     0     0]
 [    1     0   382  2944   169     0     0     0     0     0]
 [   16     0    23   268   370     0     0     0     0     0]
 [   12     0    10   211   350     0     0     0     0     0]
 [    0     0     0   373     5     0     0     0     0     0]
 [    3     0    26    15     0     0     0     0     0     0]]
              precision    recall  f1-score   support

           0       0.63      0.39      0.49     37000
           1       0.00      0.00      0.00     18871
           2       0.38      0.36      0.37     11132
           3       0.16      0.84      0.26      6062
           4       0.07      0.28      0.11      4089
           5       0.00      0.00      0.00 

c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\phil\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif